# Computing NPV of Swap Trades

In this session we will look at how to calculate Net Present Value (NPV) of swap trades using OpenRedukti.

### Overview of steps

The process involves following steps:

* First we will construct zero curves using the steps described in the previous session.
* Then we will register various data structures with the Redukti GRPC server
* Finally we will construct some swap trades and compute their NPVs

### Checking Basics

In [2]:
from redukti.library import ServerCommand, GRPCAdapter, LocalAdapter

In [4]:
# adapter = GRPCAdapter('localhost:9001')
adapter = LocalAdapter('/Software/redukti/scripts/pricing.lua')
server_command = ServerCommand(adapter)

In [5]:
server_command.hello('hello redukti')

'hello redukti'

The server has echoed back our input so that means it is running.

### Loading Market Data 

We can now load the curve definitions and par rates into memory structures using a library function provided by redukti.

In [6]:
from redukti.library import MarketData
from redukti import enums_pb2 as enums
import redukti

In [7]:
market_data = MarketData(redukti.Date.dmy(11,12,2012), enums.CURVE_GROUP_C) # business date is 11th Dec 2012

In [8]:
market_data.read_curvedefinitions('../testdata/20121211/curve_definitions.csv')

In [9]:
market_data.read_parcurves('../testdata/20121211/par_rates.csv')

We can also load fixings data. These are available in a file.

In [10]:
market_data.read_fixings('../testdata/20121211/fixings.csv')

### Building Curves

We can now request the server to bootstrap the curves. 

In [11]:
server_command.build_curves(market_data)

Building curves, please wait.
Curve 7 created
Curve 8 created
Curve 9 created
Curve 10 created


### Register Market Data With Redukti Server

Now that our curves are ready we can register those with the Redukti Server:

We need to make a list of the curves that will be used for estimating forward rates. In our case these are curves with ids 8,9, and 10

In [12]:
forward_curves = [8,9,10]

We also need a list of curves that will be used for discounting cashflows. In our case this is the EONIA curve which has id 7.

In [13]:
discount_curves = [7]

We have all the market data we need for computing NPVs, but we must load these into the Redukti Server. The Redukti Server provides apis for loading curve definitions, zero rates and fixings. These are wrapped in a method named `register_market_data()` which we will invoke now.

In [14]:
server_command.register_market_data(market_data, forward_curves, discount_curves)

Registering curve definitions
Registering zero curves
Registering fixings


### Curve Mappings

The next thing we need to do is define curve mappings. Curve mappings provide the glue between the pricing routines and the actual curves. When you create cashflows in OpenRedukti they do not reference any curves. So how will the pricing routines know which curves to use? The pricing routines ask a Curve Provider to find the best possible curve for a given currency, index family and tenor; as well as a qualifier to say whether the curve is needed for computing forward rates or for discounting. The Curve Provider is a simple lookup manager that uses the curve mappings you provide to find the best curve to use.

In our example here we have four curves. These are:

* EUR EONIA
* EUR EURIBOR 
* EUR EURIBOR 3M
* EUR EURIBOR 6M

What we want is that EONIA curve should always be used for discounting.
Secondly we want the forward rates to be computed using EONIA curve for OIS swaps, but for regular EUR swaps we want to use the most specific EURIBOR curve we have. That is, if the swap is calculating interest every three months, then we would like to use the 3M curve, if the calculation frequency is six months, then we would prefer using the 3M curve, and otherwise we will use the more generic EURIBOR curve.

One of the things to bear in mind is that we registered the curves such that the EURIBOR curves (identifier 8,9,10) were registered as Forward curves and the EONIA curve was registered as Discount curve. By default this means that a lookup for discounting for OIS swaps will result in the EONIA curve. Also it means that lookups for forward curves when index is EURIBIR will automatically find the correct forward curve.

We there have two scenarios remaining.

* What should be the forward curve for OIS swaps, i.e. index is EONIA
* What should be the discount curve for EURIBOR swaps, i.e. index is EURIBOR

This is what we will try to setup now.

In [15]:
from redukti import valuation_pb2 as valuation
curve_mappings = []

curve_mapping = valuation.CurveMapping()
# If forward curve for EONIA is requested
curve_mapping.from_id.type = enums.PRICING_CURVE_TYPE_FORWARD
curve_mapping.from_id.currency = enums.EUR
curve_mapping.from_id.index_family = enums.EONIA
# we should supply the EONIA discount curve
curve_mapping.to_id.type = enums.PRICING_CURVE_TYPE_DISCOUNT
curve_mapping.to_id.currency = enums.EUR
curve_mapping.to_id.index_family = enums.EONIA
curve_mappings.append(curve_mapping)

In [16]:
# Now add mapping for EURIBOR discount to EONIA discount
curve_mapping = valuation.CurveMapping()
# If forward curve for EONIA is requested
curve_mapping.from_id.type = enums.PRICING_CURVE_TYPE_DISCOUNT
curve_mapping.from_id.currency = enums.EUR
curve_mapping.from_id.index_family = enums.EURIBOR
# we should supply the EONIA discount curve
curve_mapping.to_id.type = enums.PRICING_CURVE_TYPE_DISCOUNT
curve_mapping.to_id.currency = enums.EUR
curve_mapping.to_id.index_family = enums.EONIA
curve_mappings.append(curve_mapping)

In [17]:
# Lets look at the curve mappings we created.#
curve_mappings

[from_id {
   type: PRICING_CURVE_TYPE_FORWARD
   currency: EUR
   index_family: EONIA
 }
 to_id {
   type: PRICING_CURVE_TYPE_DISCOUNT
   currency: EUR
   index_family: EONIA
 }, from_id {
   type: PRICING_CURVE_TYPE_DISCOUNT
   currency: EUR
   index_family: EURIBOR
 }
 to_id {
   type: PRICING_CURVE_TYPE_DISCOUNT
   currency: EUR
   index_family: EONIA
 }]

The next step is to register these curve mappings to the Redukti Valuation Server

In [18]:
server_command.register_curve_mappings(enums.CURVE_GROUP_C, curve_mappings)

Registering curve mappings


### Create cashflows for a Vanilla Swap trade

Now that our valuation server is ready to compute swapo valuations, we need an IRS swap we can submit to it. We have to do this in two steps:

* First we construct protobuf cashflow definitions
* Then we can submit the cashflows for valuation

The redukti library provides some help routines to do this. Here we will construct a 10 year swap that pays fixed and receives floating every six months.

In [19]:
from redukti.library import build_vanilla_swap

In [20]:
cashflows = build_vanilla_swap(1000000, redukti.Date.dmy(21,8,2019), redukti.Date.dmy(21,8,2029), 'EUR_EURIBOR_6M', 0.025, -1.0)

Above we specified a template called `EUR_EURIBOR_6M` to be used so that we didn't have to fill in all fields. Fixed rate was specified as `0.025`. The notional is `1m`. The last `-1.0` is saying that the fixed leg should be negative as it represents payer, and floating leg should be opposite sign.

You can look at the generated cashflows.

In [21]:
cashflows

streams {
  cashflows {
    simple {
      currency: EUR
      amount: 12777.777777777777
      payment_date: 43882
      discounting_index_family: EONIA
    }
  }
  cashflows {
    simple {
      currency: EUR
      amount: 12638.888888888889
      payment_date: 44064
      discounting_index_family: EONIA
    }
  }
  cashflows {
    simple {
      currency: EUR
      amount: 12777.777777777777
      payment_date: 44249
      discounting_index_family: EONIA
    }
  }
  cashflows {
    simple {
      currency: EUR
      amount: 12569.444444444443
      payment_date: 44431
      discounting_index_family: EONIA
    }
  }
  cashflows {
    simple {
      currency: EUR
      amount: 12777.777777777777
      payment_date: 44613
      discounting_index_family: EONIA
    }
  }
  cashflows {
    simple {
      currency: EUR
      amount: 12569.444444444443
      payment_date: 44795
      discounting_index_family: EONIA
    }
  }
  cashflows {
    simple {
      currency: EUR
      amount: 12777

### Submitting cashflows for valuation

In this step we will submit the cashflows to the OpenRedukti Valuation server for valuation. For this we need to create a PricingContext object wchich we can obtain using our MarketData object. This sets up some default values that are fine for this requirement.

In [22]:
pricing_context = market_data.pricing_context()
pricing_context

as_of_date: 41254
payment_cutoff_date: 41254
derivative_order: 2
curve_group: CURVE_GROUP_C

In the request for valuation we need to supply the pricing_context and the cashflows objects.

In [23]:
valuation_reply = server_command.get_valuation(pricing_context, cashflows)

Requesting valuation


In [24]:
valuation_reply.result.valuations

{0: 35048.86391002609}

The valuation results also include zero rate sensitivities.

### Valuation of OIS swap trade

Here we will ask an OIS trade to be valued

In [25]:
cashflows = build_vanilla_swap(1000000, redukti.Date.dmy(21,8,2019), redukti.Date.dmy(21,8,2029), 'EUR_EONIA_1D', 0.025, -1.0)

In [26]:
cashflows

streams {
  cashflows {
    simple {
      currency: EUR
      amount: 25416.666666666664
      payment_date: 44064
      discounting_index_family: EONIA
    }
  }
  cashflows {
    simple {
      currency: EUR
      amount: 25347.222222222223
      payment_date: 44431
      discounting_index_family: EONIA
    }
  }
  cashflows {
    simple {
      currency: EUR
      amount: 25347.222222222223
      payment_date: 44795
      discounting_index_family: EONIA
    }
  }
  cashflows {
    simple {
      currency: EUR
      amount: 25347.222222222223
      payment_date: 45159
      discounting_index_family: EONIA
    }
  }
  cashflows {
    simple {
      currency: EUR
      amount: 25416.666666666664
      payment_date: 45525
      discounting_index_family: EONIA
    }
  }
  cashflows {
    simple {
      currency: EUR
      amount: 25347.222222222223
      payment_date: 45890
      discounting_index_family: EONIA
    }
  }
  cashflows {
    simple {
      currency: EUR
      amount: 25347

In [27]:
valuation_reply = server_command.get_valuation(pricing_context, cashflows)

Requesting valuation


In [28]:
valuation_reply.result.valuations

{0: 17093.513689989544}